In [2]:
import os 

import copy

import tensorflow as tf
from tensorflow import keras
from keras import Input, Model, Sequential
from keras.layers import Activation
from keras.layers import Conv2D, MaxPooling2D,ZeroPadding2D, BatchNormalization, Dense, Flatten, Reshape , Dropout

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split



In [3]:
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm() 

  Created wheel for gputil: filename=GPUtil-1.4.0-cp36-none-any.whl size=7411 sha256=b2894ed3421bfd25529404cf7528481421513950652e6663e32ef356e2736e6f
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil
Gen RAM Free: 12.7 GB  | Proc size: 370.9 MB
GPU RAM Free: 15079MB | Used: 0MB | Util   0% | Total 15079MB


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
def get_data(file): 

    data = pd.read_csv(file)

    feat_raw = data['quizzes']
    label_raw = data['solutions']

    feat = []
    label = []

    for i in feat_raw:
    
        x = np.array([int(j) for j in i]).reshape((9,9,1))
        feat.append(x)
    
    feat = np.array(feat)
    feat = feat/9
    feat -= .5    
    
    for i in label_raw:
    
        x = np.array([int(j) for j in i]).reshape((81,1)) - 1
        label.append(x)   
    
    label = np.array(label)
    
    del(feat_raw)
    del(label_raw)    

    x_train, x_test, y_train, y_test = train_test_split(feat, label, test_size=0.2, random_state=42)
    
    return x_train, x_test, y_train, y_test , data

In [6]:
x_train, x_test, y_train, y_test , data = get_data('drive/MyDrive/Colab Notebooks/SUDOKU/sudoku.csv')

In [7]:
print("dd")
print(data)
print("next")
print(x_train.shape)

dd
                                                  quizzes                                          solutions
0       0043002090050090010700600430060020871900074000...  8643712593258497619712658434361925871986574322...
1       0401000501070039605200080000000000170009068008...  3461792581875239645296483719658324174729168358...
2       6001203840084590720000060050002640300700800069...  6951273841384596727248369158512647392739815469...
3       4972000001004000050000160986203000403009000000...  4972583161864397252537164986293815473759641828...
4       0059103080094030600275001000300002010008200070...  4659123781894735623275681497386452919548216372...
...                                                   ...                                                ...
999995  3000280000290000300054001077402030980086070031...  3175289464291768356854391277462135989586472131...
999996  0030006000040860057000009409350407208067200502...  5234976811942863757685139429356417288167294532...
999997  00035082

In [8]:
def get_model():

    model = keras.models.Sequential()

    model.add(Conv2D(64, kernel_size=(3,3), activation='relu', padding='same', input_shape=(9,9,1)))
    model.add(BatchNormalization())
    model.add(Conv2D(64, kernel_size=(3,3), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(Conv2D(128, kernel_size=(1,1), activation='relu', padding='same'))

    model.add(Flatten())
    model.add(Dense(81*9))
    model.add(Reshape((-1, 9)))
    model.add(Activation('softmax'))
    return model

In [50]:
rows, cols = 9, 9
def create_convnet():
    input_shape = Input(shape=(rows, cols, 1))

    tower_1 = Conv2D(512, (3,3), padding='valid', strides=(3,3), activation='relu')(input_shape)
    #tower_1 = MaxPooling2D((1, 11), strides=(1, 1),  padding='same')(tower_1)
    tower_1 = BatchNormalization()(tower_1)
    tower_1 = Flatten()(tower_1)

    tower_2 = Conv2D(512, (9, 1), padding='valid', strides=(1, 9), activation='relu')(input_shape)
    tower_2 = BatchNormalization()(tower_2)
    tower_2 = Flatten()(tower_2)

    tower_3 = Conv2D(512, (1, 9), padding='valid', strides=(9, 1), activation='relu')(input_shape)    
    tower_3 = BatchNormalization()(tower_3)
    tower_3 = Flatten()(tower_3)

    merged = keras.layers.concatenate([tower_1, tower_2, tower_3], axis=1)
    #merged = Conv2D(128, kernel_size=(3,3), activation='relu', padding='same')(merged)

    merged = Flatten()(merged)
    print(merged.shape)

    #dense = Dense((2048))(merged)
    #dense = Conv2D(128, kernel_size=(3,3), activation='relu', padding='same')(merged)
    #dense = Conv2D(512, (3, 3), padding='same', activation='relu')(dense)

    #ajouter dense/conv ou deux entre taille du merge et taille de out  
    out = Dense(81*9)(merged)
    print(out.shape)
    out = Reshape((-1, 9))(out)
    out = tf.keras.layers.Activation('softmax')(out)

    
    model = Model(input_shape, out)
    
    
    return model

In [51]:
create_convnet()
print(512*9)

(None, 5632)
(None, 729)
4608


In [11]:
def get_model_Dense():

    model = keras.models.Sequential()

    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Dense(256, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Dense(128, activation='relu'))

    model.add(Flatten())
    model.add(Dense(81*9))
    model.add(Reshape((-1, 9)))
    model.add(Activation('softmax'))
    return model

In [12]:
def create_ResNet():
    model = tf.keras.applications.ResNet50(
        include_top=True, weights=None , input_shape=None,  input_tensor=None, classes=9
    )
    return model

In [ ]:
#model = get_model()
#model = get_model_Dense()
model = create_convnet()
#model = create_ResNet()

adam = keras.optimizers.Adam(lr=.0001)
model.compile(loss='sparse_categorical_crossentropy', optimizer=adam)
#model.summary()

model.fit(x_train, y_train, batch_size=128, epochs=20)

In [73]:
model.evaluate(x_test, y_test)

6250/6250 [==============================] - 10s 2ms/step - loss: 0.4249


0.4249066412448883

In [ ]:
model.summary()

In [59]:
model.save('drive/MyDrive/Colab Notebooks/SUDOKU/my_model/my_model.h5') 

In [ ]:
tf.keras.models.save_model(
    model, 'drive/MyDrive/Colab Notebooks/SUDOKU/my_model2', overwrite=True, include_optimizer=True, save_format="model",
    signatures=None, options=None, save_traces=True
)

In [60]:
def norm(a):
    
    return (a/9)-.5

In [61]:
def denorm(a):
    
    return (a+.5)*9

In [62]:
def inference_sudoku(sample):
    
    '''
        This function solve the sudoku by filling blank positions one by one.
    '''
    
    feat = copy.copy(sample)
    
    while(1):
    
        out = model.predict(feat.reshape((1,9,9,1)))  
        out = out.squeeze()

        pred = np.argmax(out, axis=1).reshape((9,9))+1 
        prob = np.around(np.max(out, axis=1).reshape((9,9)), 2) 
        
        feat = denorm(feat).reshape((9,9))
        mask = (feat==0)
     
        if(mask.sum()==0):
            break
            
        prob_new = prob*mask
    
        ind = np.argmax(prob_new)
        x, y = (ind//9), (ind%9)

        val = pred[x][y]
        feat[x][y] = val
        feat = norm(feat)
    
    return pred

In [63]:
def test_accuracy(feats, labels):
    
    correct = 0
    
    for i,feat in enumerate(feats):
        
        pred = inference_sudoku(feat)
        
        true = labels[i].reshape((9,9))+1
        
        if(abs(true - pred).sum()==0):
            correct += 1
        
    print(correct/feats.shape[0])

In [64]:
test_accuracy(x_test[:100], y_test[:100])

0.44


In [65]:
def solve_sudoku(game):
    
    game = game.replace('\n', '')
    game = game.replace(' ', '')
    game = np.array([int(j) for j in game]).reshape((9,9,1))
    game = norm(game)
    game = inference_sudoku(game)
    return game

In [66]:
game = '''
          0 8 0 0 3 2 0 0 1
          7 0 3 0 8 0 0 0 2
          5 0 0 0 0 7 0 3 0
          0 5 0 0 0 1 9 7 0
          6 0 0 7 0 9 0 0 8
          0 4 7 2 0 0 0 5 0
          0 2 0 6 0 0 0 0 9
          8 0 0 0 9 0 3 0 5
          3 0 0 8 2 0 0 1 0
      '''

game = solve_sudoku(game)

print('solved puzzle:\n')
print(game)

solved puzzle:

[[4 8 9 5 3 2 7 8 1]
 [7 1 3 4 8 6 5 9 2]
 [5 8 2 9 1 7 6 3 4]
 [2 5 8 3 4 1 9 7 6]
 [6 3 1 7 5 9 4 2 8]
 [9 4 7 2 6 8 1 5 3]
 [1 2 5 6 7 3 8 4 9]
 [8 7 2 1 9 4 3 6 5]
 [3 9 4 8 2 5 2 1 7]]


In [67]:
np.sum(game, axis=1)

array([47, 45, 45, 45, 45, 45, 45, 45, 41])